# Balance de Masa Superficial - Modelo de Lliboutry
### Autor: Rubén Basantes

### Configuración de entorno de trabajo

In [68]:
system("git clone https://github.com/jpanimboza/Repo_prueba_Mass_Balance.git", intern = TRUE)

# Cambia al directorio
setwd("Repo_prueba_Mass_Balance")

character(0)

### Instalación y carga de librerías

In [69]:
suppressPackageStartupMessages({
  source("Libraries.R")    
})

### Carga de datos

In [70]:
rm(list = ls())
rawData <- "data/bmantisana15.txt"
shapes_path <- "data/"
m <- 50 # range for the calculation of the balance by the weighted area method
Resolution <- 100 # It is the resolution of the raster to be used. Cell size = mxm units
N.Obs <- 6
crsID <- "EPSG:32717"
dem <- "data/Antisana_ALOS.tif"
summit <- 5700

#### Crear un cuaderno de trabajo

In [71]:
wb <- createWorkbook()

### Balance de masa según el modelo de Lliboutry

In [72]:
source("funciones/Lliboutry_NL.R")
NLMR <- NLM(rawData, shapes_path, dem, Resolution, crsID, N.Obs, summit)

Reading layer `Antisana15a_1999' from data source 
  `/kaggle/working/Repo_prueba_Mass_Balance/Repo_prueba_Mass_Balance/Repo_prueba_Mass_Balance/Repo_prueba_Mass_Balance/Repo_prueba_Mass_Balance/Repo_prueba_Mass_Balance/data/Antisana15a_1999.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 1 field
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 816781.9 ymin: 9946475 xmax: 818009.4 ymax: 9947552
Projected CRS: WGS 84 / UTM zone 17S


In [73]:
#{r}
e_obs_mod <- NLMR$e_obs_mod
Nyear <- NLMR$Nyear
bit_LM <- NLMR$bit_LM
bit_LMf <- NLMR$bit_LMf
bo_t <- NLMR$bo_t
beta_it <- NLMR$beta_it
brick_raw <-NLMR$brick_raw
rawData <- NLMR$rawData
shap <- NLMR$shap
shape_list <- NLMR$shape_list
ai_df <- NLMR$ai_df
XY_coords <- NLMR$XY_coords
e_obs_mod_sd.tot <- sd(unlist(e_obs_mod[-c(1:3)]), na.rm=T)
#{r}

#### Verificar normalidad con la prueba de 